In [0]:
import pandas as pd

In [0]:
posts_df = pd.read_parquet("/Volumes/workspace/default/raw/reddit_posts.parquet")
posts_df.head()

,id,title,author,score,num_comments,created_utc,selftext,subreddit
0,1mniom8,Reddit will block the Internet Archive,MarvelsGrantMan136,15679,1232,1.754932e+09,NaN,technology
1,1mn95cp,A massive Wyoming data center will soon use 5x...,DJMagicHandz,24928,1520,1.754907e+09,NaN,technology
2,1mnghv6,RFK Jr. wants a wearable on every American — t...,DonkeyFuel,3133,436,1.754927e+09,NaN,technology
3,1mngvle,GitHub is no longer independent at Microsoft a...,TheOneByron,1173,85,1.754928e+09,NaN,technology
4,1mnekv7,"Ford Aims for Revolution With $30,000 Electric...",DonkeyFuel,1391,337,1.754923e+09,NaN,technology


In [0]:
comments_df = pd.read_parquet("/Volumes/workspace/default/raw/reddit_comments.parquet")
comments_df.head()

,id,author,body,score,created_utc,parent_id,link_id
0,n853d05,JamesTiberiusCrunk,Entirely because they want to sell post data t...,7777,1.754932e+09,t3_1mniom8,t3_1mniom8
1,n852wwp,FollowingFeisty5321,"Outrageous, especially with how often posts, t...",7232,1.754932e+09,t3_1mniom8,t3_1mniom8
2,n85718n,MyLovelyMan,It’s no coincidence that in this year:\n- Inte...,2410,1.754933e+09,t3_1mniom8,t3_1mniom8
3,n853ihn,extremetolerance2013,"So to protect this from AI, we must close it f...",1435,1.754932e+09,t3_1mniom8,t3_1mniom8
4,n855qr7,ParticularBeing6686,Someone should make a browser extension to cro...,485,1.754933e+09,t3_1mniom8,t3_1mniom8


In [0]:
posts_profiling_report = pd.DataFrame({
    "Column Name": posts_df.columns,
    "Data Type": posts_df.dtypes.values,
    "Null Count": posts_df.isnull().sum().values,
    "Null %": (posts_df.isnull().sum() / len(posts_df) * 100).values.round(2),
    "Distinct Values": [posts_df[col].nunique() for col in posts_df.columns],
    "Most Frequent Value": ["All unique" if posts_df[col].nunique(dropna=True) == len(posts_df[col].dropna())
    else ", ".join(map(str, posts_df[col].mode().tolist())) if not posts_df[col].mode().empty
    else "None"
    for col in posts_df.columns]
})


In [0]:
posts_profiling_report.to_csv("posts_profiling_report.csv", index=False)

In [0]:
comments_profiling_report = pd.DataFrame({
    "Column Name": comments_df.columns,
    "Data Type": comments_df.dtypes.values,
    "Null Count": comments_df.isnull().sum().values,
    "Null %": (comments_df.isnull().sum() / len(comments_df) * 100).values.round(2),
    "Distinct Values": [comments_df[col].nunique() for col in comments_df.columns],
    "Most Frequent Value": ["All unique" if comments_df[col].nunique(dropna=True) == len(comments_df[col].dropna())
    else ", ".join(map(str, comments_df[col].mode().tolist())) if not comments_df[col].mode().empty
    else "None"
    for col in comments_df.columns]
})

In [0]:
comments_profiling_report.to_csv("comments_profiling_report.csv", index=False)

In [0]:
# Drop the selftext column
cleaned_posts_df = posts_df.drop(columns=['selftext'])

# Convert created_utc to datetime
#cleaned_posts_df['created_utc'] = pd.to_datetime(posts_df['created_utc'], unit='s')

In [0]:
!pip install ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00


In [0]:
import ftfy
import unicodedata

In [0]:
cleaned_posts_df['title'] = posts_df['title'].apply(
    lambda x: unicodedata.normalize("NFKD", ftfy.fix_text(x)).encode("ascii", "ignore").decode("ascii"))

In [0]:
cleaned_posts_df.to_parquet("/Volumes/workspace/default/raw/cleaned_posts.parquet", index=False, engine="pyarrow")

In [0]:
# Remove rows where 'author' is null
cleaned_comments_df = comments_df.dropna(subset=['author'])

# Clean encoding issues in 'body'
cleaned_comments_df['body'] = cleaned_comments_df['body'].apply(
    lambda x: unicodedata.normalize("NFKD", ftfy.fix_text(x)).encode("ascii", "ignore").decode("ascii"))

#cleaned_comments_df['body'] = cleaned_comments_df['body'].str.replace(',', '', regex=False)
# Remove multiple characters from 'body'
cleaned_comments_df['body'] = cleaned_comments_df['body'].str.replace(
    r'[,\-:>"\n\r]', '', regex=True)

/tmp/ipython-input-384984792.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_comments_df['body'] = cleaned_comments_df['body'].apply(
/tmp/ipython-input-384984792.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_comments_df['body'] = cleaned_comments_df['body'].str.replace(


In [0]:
cleaned_comments_df.to_parquet("/Volumes/workspace/default/raw/cleaned_comments.parquet", index=False, engine="pyarrow")